In [117]:
import unittest
import numpy
import importlib
import sys
sys.path.append('../')
from src import basis_class

# Reload the module to apply changes
importlib.reload(basis_class)
from src.basis_class import Basis

In [115]:
class BasisTestCase(unittest.TestCase):
    
    def test_get_correct_dimensions(self):
        n = 2
        basis_class = Basis(2)
        result = basis_class.dimension
        self.assertEqual(result, n)
    
    def test_only_even_dimensions(self):
        n = 3
        with self.assertRaises(Exception) as context:
            Basis(n)  # This should raise the exception

        
        self.assertEqual(str(context.exception), "Only positive even integers allowed!")

    def test_only_positive_values(self):
        n = -4
        with self.assertRaises(Exception) as context:
            Basis(n)
        
        self.assertEqual(str(context.exception), "Only positive even integers allowed!")
        

        

In [116]:
unittest.main(argv=[''], exit=False)

.FF
FAIL: test_only_even_dimensions (__main__.BasisTestCase.test_only_even_dimensions)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\waqas\AppData\Local\Temp\ipykernel_11692\1908930920.py", line 11, in test_only_even_dimensions
    with self.assertRaises(Exception) as context:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: Exception not raised

FAIL: test_only_positive_values (__main__.BasisTestCase.test_only_positive_values)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\waqas\AppData\Local\Temp\ipykernel_11692\1908930920.py", line 19, in test_only_positive_values
    with self.assertRaises(Exception) as context:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: Exception not raised

----------------------------------------------------------------------
Ran 3 tests in 0.011s

FAILED (failures=2)
